<a href="https://colab.research.google.com/github/hangatzu2017/Natural-Language-to-SQL/blob/main/SQL-CODER-ESP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primo esperimento (al momento totalmente in lingua inglese) utilizzando il modello opensource SQL CODER di Defog.

In [1]:
!nvidia-smi

Fri Dec 22 15:52:09 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install -U langchain
!pip install -U ctransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 71.1 MB/s eta 0:00:00


In [4]:
from langchain.llms import CTransformers

# Configurazione parametri base LLM
config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 2048, 'temperature':0, 'gpu_layers':50}

# Utilizziamo il modello quantizzato a 4bit (formato GGUF)
llm = CTransformers(model='TheBloke/sqlcoder-7B-GGUF',model_file="sqlcoder-7b.Q8_0.gguf",gpu_layers=50, config=config)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

sqlcoder-7b.Q8_0.gguf:   0%|          | 0.00/7.70G [00:00<?, ?B/s]

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Prompt template
prompt_template = """### Instructions:
Your task is convert a question into a SQL query, given a Postgres database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use Table Aliases** to prevent ambiguity. For example, `SELECT table1.col1, table2.col1 FROM table1 JOIN table2 ON table1.id = table2.id`.
- When creating a ratio, always cast the numerator as float

### Input:
Generate a SQL query that answers the questioq `{question}`.
This query will run on a database whose schema is represented in this string:
{tables}

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question `{question}`:
```sql
"""

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["tables", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
tables = """
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id
"""

question = "What products has the biggest fall in sales in 2022 compared to 2021? Give me the product name, the sales amount in both years, and the difference."


In [9]:
response = llm_chain.invoke({"tables":tables, "question":question})

print(response['text'])

WITH year_2022_sales AS (SELECT p.product_id, SUM(s.quantity) AS total_quantity_year_2022 FROM products p JOIN sales s ON p.product_id = s.product_id WHERE EXTRACT(YEAR FROM sale_date) = 2022 GROUP BY p.product_id), year_2021_sales AS (SELECT p.product_id, SUM(s.quantity) AS total_quantity_year_2021 FROM products p JOIN sales s ON p.product_id = s.product_id WHERE EXTRACT(YEAR FROM sale_date) = 2021 GROUP BY p.product_id), product_difference AS (SELECT y22.product_id, y22.total_quantity_year_2022 - coalesce(y21.total_quantity_year_2021, 0) AS total_quantity_difference FROM year_2022_sales y22 LEFT JOIN year_2021_sales y21 ON y22.product_id = y21.product_id) SELECT p.name, pd.total_quantity_difference, y22.total_quantity_year_2022, coalesce(y21.total_quantity_year_2021, 0) AS total_quantity_year_2021 FROM products p JOIN product_difference pd ON p.product_id = pd.product_id LEFT JOIN year_2022_sales y22 ON p.product_id = y22.product_id LEFT JOIN year_2021_sales y21 ON p.product_id = y21